## Build and track our portfolio based on our trades.

### Doesn't have any concept of buy and sell just yet.  only that a position is bought and held

In [ ]:
import pandas as pd
import yfinance as yf
import plotly.graph_objects as go
import pylab

Spoofing the data for now. Could come from xlsx or  pull from broker if there is an API

In [ ]:
# data = {'tickers': ['BTC-AUD', 'BOE.AX', 'PDN.AX', 'ETH-AUD', 'BOE.AX'] ,
#         'prices': [10000, 0.34, 0.40, 300.0, 0.4 ],
#         'amounts': [0.01, 3000, 2100, 0.11, -2000],  # negative amounts indicates a sell
#         'dates': ['2021-01-01', '2021-01-01', '2021-01-01', '2021-01-01', '2021-05-01'],
#         'type': ['crypto', 'equity', 'equity', 'crypto', 'equity' ],
#         'commodity': [None, 'U', 'U', None, 'U']}  # Spoofing for now. Could come from xlsx or  pull from broker if there is an API

We load the data into a pandas dataframe

Not Spoofing any more.  we're taking the data from a subset of our trades xlsx spreadsheet. 

In [ ]:
# trade_data = pd.DataFrame.from_dict(data)

In [ ]:
# trade_data.head()

In [ ]:
trade_data = pd.read_csv('data/trades.csv', delimiter=',')

In [ ]:
trade_data.head()

lets look for duplicate tickers meaning we have either bought or sold the same ticker more than once

In [ ]:
type(trade_data['tickers'].unique())

So we wish to see how each trade has done on a daily basis.  We will use yfinance for now, could get from any number of APIs 

If you haven't already, `pip install yfinance` in a terminal

Lets define some constants which we will use to setup the scope of our analysis

In [ ]:
PERIOD = '1D' # Daily
END = '2021-06-28' #  could use datetime.now

In [ ]:
quotes = [] # empty list for storing quotes

for ticker, date in zip(trade_data['tickers'], trade_data['dates'] ):
    _quote = yf.Ticker(ticker)
    quotes.append(_quote.history(period=PERIOD, start=date, end=END))

<div class="alert alert-danger" role="alert">
  ^^^ We  can see a few missing. Need to fix these later. 
</div>

In [ ]:
quotes[0].head()

----------
Lets open some candlestick charts [Helpful link](https://coderzcolumn.com/tutorials/data-science/candlestick-chart-in-python-mplfinance-plotly-bokeh)  Lets use Plotly for now.  `pip install plotly`

To use Plotly need to install [Node.js](https://nodejs.org/en/download/current/)    Do this by typing into a terminal

`conda install jodejs`   
followed by:   
`jupyter labextension install @jupyterlab/plotly-extension`   
----------------

Now lets make a little dropdown box for us to chose the ticker we want to see below in the plot    
`conda install ipywidgets`    
`jupyter labextension install @jupyter-widgets/jupyterlab-manager`


In [ ]:
from ipywidgets import interact, interact_manual

In [ ]:
type(trade_data['tickers'])

In [ ]:
@interact()
def make_plot(ticker=trade_data['tickers']):

    ii = list(trade_data['tickers']).index(ticker)
    candlestick = go.Candlestick(x=quotes[ii].index,
                                open=quotes[ii]['Open'],
                                high=quotes[ii]['High'],
                                low=quotes[ii]['Low'],
                                close=quotes[ii]['Close'])

    fig = go.Figure(data=[candlestick])
    fig.update_layout(xaxis_rangeslider_visible=False,
                     title=trade_data['tickers'][ii],
                     template='plotly_dark')

    fig.show()

Now we need to generate our port folio based on the ticker data.  Dealing with datetimes is always painful

In [ ]:
from datetime import datetime
_dates = []

for date in trade_data['dates']:
    _dates.append(datetime.strptime(date, '%Y-%m-%d').date())


date_range = pd.date_range(start=min(_dates), end=END, freq='D')

In [ ]:
date_range

In [ ]:
df = None
portfolio = date_range.to_frame()
frames = [portfolio]

for ii in range(len(trade_data['tickers'])):
    print(ii)
    frames.append(quotes[ii]['Close'] * trade_data['amounts'][ii])

df = pd.concat(frames, axis=1)
df = df.drop(df.columns[0], axis=1)

df.columns = trade_data['tickers']

portfolio = df

In [ ]:
portfolio.fillna(method='ffill', inplace=True)  ## forward fill.  so on weekends we carry forward Friday's value

In [ ]:
portfolio['Total'] = portfolio.sum(axis=1)  # add to total as a column

In [ ]:
portfolio

now lets merge duplicate columns wich represent multiple trades 

In [ ]:
portfolio = portfolio.groupby(portfolio.columns, axis=1).sum()

In [ ]:
import plotly.express as px

fig = px.line(portfolio, width=1600, height=800)
fig.update_layout(xaxis_rangeslider_visible=False,
                 title='Portfolio Value',
                 template='plotly_dark')
fig.show()

In [ ]:
# d = trade_data.groupby(['type']).sum()
# pylab.pie(d,  shadow = True, autopct='%1.1f%%', textprops={'color':"w"});
# d

In [ ]:
portfolio.count()[0]-1

In [ ]:
@interact()
def plot_pie(day=(0, portfolio.count()[0]-1)):
    vec = portfolio.iloc[day].to_numpy()[:-1]
    pylab.pie(vec, labels=portfolio.columns[:-1], shadow = True, autopct='%1.1f%%', textprops={'color':"w"});

In [ ]:
portfolio.corr()

In [ ]:
import plotly.express as px

fig = px.imshow(portfolio.corr(), width=1600*2, height=800*2)
fig.update_layout(
    title= 'Pearson Correlation Coefficient')
fig.show()